In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict,namedtuple
import bisect

In [2]:
test_df = pd.read_csv('user_log_testset.csv', sep='|', dtype={'uid': str, 'jobNo': str})
grouped_df = test_df[test_df.action != 'viewCust'].groupby(['uid', 'jobNo', 'action']).size().reset_index(name='count')
pivot_df = grouped_df.pivot_table(index=['uid', 'jobNo'], columns='action', values='count').fillna(0)
pivot_df = pivot_df.reindex_axis(['viewJob', 'saveJob', 'applyJob'], axis=1)
pivot_df.head()

action                 viewJob  saveJob  applyJob
uid           jobNo                              
1005022347269 1000350      1.0      0.0       NaN
              1013754      1.0      0.0       NaN
              1143308      1.0      0.0       NaN
              1370665      2.0      0.0       NaN
              1544933      1.0      0.0       NaN

In [3]:
user_applyrate = pd.read_csv('tables/user_applyrate.csv',index_col = 0)
user_applyrate.head()

,view,apply,rate
1597727834569,126,1,0.007937
1580547972837,7,1,0.142857
1580547965728,2,1,0.500000
1580547965729,5,2,0.400000
1580547972832,6,0,0.000000


In [4]:
job_applyrate = pd.read_csv('tables/job_applyrate.csv',index_col = 0)
job_applyrate.head()

,apply,view,rate
8688841,91.0,728.0,0.125000
8890700,58.0,489.0,0.118609
5333263,19.0,114.0,0.166667
7695902,135.0,361.0,0.373961
8071298,180.0,2448.0,0.073529


In [5]:
b = [int(i) in user_applyrate.index for i in [i[0] for i in pivot_df.index]]
exist_uid = [i for i in set([i[0] for i in pivot_df.index]) if int(i) in user_applyrate.index]

In [6]:
view1 = pd.read_csv('tables/view1times.csv',index_col = 0)
view2 = pd.read_csv('tables/view2times.csv',index_col = 0)
view3 = pd.read_csv('tables/view3times.csv',index_col = 0)
view4 = pd.read_csv('tables/view4times.csv',index_col = 0)
view5 = pd.read_csv('tables/view5times.csv',index_col = 0)
view6 = pd.read_csv('tables/view6times.csv',index_col = 0)
view7 = pd.read_csv('tables/view7times.csv',index_col = 0)
view8 = pd.read_csv('tables/view8times.csv',index_col = 0)
gap = [1.0*i/20 for i in range(21)]
viewtime_list = [0,view1,view2,view3,view4,view5,view6,view7,view8]

In [6]:
def predict(index):
    uid,jobno = index
    viewtimes = int(pivot_df.loc[index].viewJob)
    try:
        apply_rate = user_applyrate.loc[int(uid)].rate
    except:
        try:
            apply_rate = job_applyrate.loc[int(jobno)].rate
        except:
            apply_rate = 0
    if viewtimes == 0:
        return pivot_df.loc[index].saveJob == 0
    elif 0 < viewtimes < 9:
        return viewtime_list[viewtimes].loc[bisect.bisect(gap,apply_rate)]['rate'] > 0.3
    else:
        return bisect.bisect(gap,apply_rate)>1

In [7]:
def predict2(index):
    uid,jobno = index
    viewtimes = int(pivot_df.loc[index].viewJob)
    try:
        apply_rate = job_applyrate.loc[int(jobno)].rate
    except:
        apply_rate = 0
    if viewtimes == 0:
        return pivot_df.loc[index].saveJob == 0
    elif 0 < viewtimes < 9:
        return viewtime_list[viewtimes].loc[bisect.bisect(gap,apply_rate)]['rate'] > 0.3
    else:
        return bisect.bisect(gap,apply_rate)>1

In [9]:
exist_dict = defaultdict(int)
for i in exist_uid:
    exist_dict[i]=1

In [25]:
def predict3(index):
    uid,jobno = index
    viewtimes = int(pivot_df.loc[index].viewJob)
    try:
        apply_rate = job_applyrate.loc[int(jobno)].rate
    except:
        apply_rate = 0
    if viewtimes == 0:
        return pivot_df.loc[index].saveJob == 0
    elif 0 < viewtimes < 9:
        if exist_dict[uid] == 1:
            return viewtime_list[viewtimes].loc[bisect.bisect(gap,apply_rate)]['rate'] > 0.21
        else:
            return viewtime_list[viewtimes].loc[bisect.bisect(gap,apply_rate)]['rate'] > 0.32
    else:
        return bisect.bisect(gap,apply_rate)>1

In [32]:
def predict4(index):
    uid,jobno = index
    viewtimes = int(pivot_df.loc[index].viewJob)
    if exist_dict[uid] == 1:
        return viewtimes > 2
    else:
        return viewtimes > 3

In [33]:
idx = pivot_df.index
print len(idx)
preds = []
for n,i in enumerate(idx):
    if n % 10000 == 0:
        print n
    preds.append(predict4(i))

2326678
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
13

In [34]:
pivot_df['applyJob'] = preds
preds_df = pivot_df.drop(['viewJob', 'saveJob'], axis=1)
preds_df.applyJob = preds_df.applyJob.astype(int)
preds_df.head()

action                 applyJob
uid           jobNo            
1005022347269 1000350         0
              1013754         0
              1143308         0
              1370665         0
              1544933         0

In [18]:
exist = pivot_df[b]

0.3

In [11]:
sum(exist['applyJob']),len(exist)

(89984, 820657)

In [12]:
sum(preds_df['applyJob']),len(preds_df)

(308134, 2326678)

0.27,0.33

In [19]:
sum(exist['applyJob']),len(exist)

(93234, 820657)

In [17]:
sum(preds_df['applyJob']),len(preds_df)

(278911, 2326678)

0.21 0.32

In [29]:
exist = pivot_df[b]
sum(exist['applyJob']),len(exist)

(127416, 820657)

In [30]:
sum(preds_df['applyJob']),len(preds_df)

(313093, 2326678)

In [37]:
preds_df.to_csv('preds_dontupload.csv', sep='|')

In [35]:
exist = pivot_df[b]
sum(exist['applyJob']),len(exist)

(113267, 820657)

In [36]:
sum(preds_df['applyJob']),len(preds_df)

(289860, 2326678)